In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_text
from sklearn.tree import plot_tree
from sklearn.datasets import load_iris
from sklearn.preprocessing import label_binarize
import pickle

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Red_Light_Camera_Violations.csv'
df = pd.read_csv(file_path)
df.head()

,INTERSECTION,CAMERA ID,ADDRESS,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,IRVING PARK AND KILPATRICK,2763.0,4700 W IRVING PARK ROA,04/09/2015,4,NaN,NaN,NaN,NaN,NaN
1,VAN BUREN AND WESTERN,2054.0,2400 W VAN BUREN STREE,04/14/2015,5,NaN,NaN,NaN,NaN,NaN
2,115TH AND HALSTED,2552.0,11500 S HALSTED STREE,04/08/2015,5,NaN,NaN,NaN,NaN,NaN
3,IRVING PARK AND KILPATRICK,2764.0,4700 W IRVING PARK ROA,04/19/2015,4,NaN,NaN,NaN,NaN,NaN
4,ELSTON AND IRVING PARK,1503.0,3700 W IRVING PARK ROA,04/23/2015,3,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957577 entries, 0 to 957576
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   INTERSECTION    957577 non-null  object 
 1   CAMERA ID       957288 non-null  float64
 2   ADDRESS         957577 non-null  object 
 3   VIOLATION DATE  957577 non-null  object 
 4   VIOLATIONS      957577 non-null  int64  
 5   X COORDINATE    907560 non-null  float64
 6   Y COORDINATE    907560 non-null  float64
 7   LATITUDE        907560 non-null  float64
 8   LONGITUDE       907560 non-null  float64
 9   LOCATION        907560 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 73.1+ MB


In [ ]:
df.isnull().sum()

INTERSECTION          0
CAMERA ID           289
ADDRESS               0
VIOLATION DATE        0
VIOLATIONS            0
X COORDINATE      50017
Y COORDINATE      50017
LATITUDE          50017
LONGITUDE         50017
LOCATION          50017
dtype: int64

In [ ]:
df.columns

Index(['INTERSECTION', 'CAMERA ID', 'ADDRESS', 'VIOLATION DATE', 'VIOLATIONS',
       'X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')

In [ ]:
df['LOCATION'].value_counts()

LOCATION
(41.86726309836901, -87.64697268601648)     7189
(41.98564979859328, -87.65533580655139)     7123
(41.903298792681944, -87.67738324739636)    7075
(41.92519388981048, -87.66829436339881)     7022
(41.94616352362556, -87.74721516897007)     6979
                                            ... 
(41.96105580370759, -87.72789120598296)      131
(41.77957043618697, -87.66429387512882)      126
(41.97525809526954, -87.75199055715778)      115
(41.801257491998776, -87.68428111564593)     107
(41.87619197518567, -87.68633785193599)        1
Name: count, Length: 297, dtype: int64

In [ ]:
df.isnull().sum()

INTERSECTION          0
CAMERA ID           289
ADDRESS               0
VIOLATION DATE        0
VIOLATIONS            0
X COORDINATE      50017
Y COORDINATE      50017
LATITUDE          50017
LONGITUDE         50017
LOCATION          50017
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957577 entries, 0 to 957576
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   INTERSECTION    957577 non-null  object 
 1   CAMERA ID       957288 non-null  float64
 2   ADDRESS         957577 non-null  object 
 3   VIOLATION DATE  957577 non-null  object 
 4   VIOLATIONS      957577 non-null  int64  
 5   X COORDINATE    907560 non-null  float64
 6   Y COORDINATE    907560 non-null  float64
 7   LATITUDE        907560 non-null  float64
 8   LONGITUDE       907560 non-null  float64
 9   LOCATION        907560 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 73.1+ MB


In [ ]:
df["CAMERA ID"] = df["CAMERA ID"].fillna(df["CAMERA ID"].mean())

In [ ]:
df["X COORDINATE"] = df["X COORDINATE"].fillna(df["X COORDINATE"].mean())

In [ ]:
df["Y COORDINATE"] = df["Y COORDINATE"].fillna(df["Y COORDINATE"].mean())

In [ ]:
df["LATITUDE"] = df["LATITUDE"].fillna(df["LATITUDE"].mean())

In [ ]:
df["LONGITUDE"] = df["LONGITUDE"].fillna(df["LONGITUDE"].mean())

In [ ]:
df["LOCATION"]=df["LOCATION"].fillna("(41.86726309836901, -87.64697268601648)")

In [ ]:
df.isnull().sum()

INTERSECTION      0
CAMERA ID         0
ADDRESS           0
VIOLATION DATE    0
VIOLATIONS        0
X COORDINATE      0
Y COORDINATE      0
LATITUDE          0
LONGITUDE         0
LOCATION          0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.columns

Index(['INTERSECTION', 'CAMERA ID', 'ADDRESS', 'VIOLATION DATE', 'VIOLATIONS',
       'X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')

In [ ]:
fig = px.histogram(df,x="INTERSECTION" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by INTERSECTION",
    xaxis_title="INTERSECTION",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="CAMERA ID" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by CAMERA ID",
    xaxis_title="CAMERA ID",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="ADDRESS" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by ADDRESS",
    xaxis_title="ADDRESS",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="VIOLATION DATE" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by VIOLATION DATE",
    xaxis_title="VIOLATION DATE",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="X COORDINATE" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by X COORDINATE",
    xaxis_title="X COORDINATE",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="Y COORDINATE" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by Y COORDINATE",
    xaxis_title="Y COORDINATE",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="LATITUDE" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by LATITUDE",
    xaxis_title="LATITUDE",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="LONGITUDE" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by LONGITUDE",
    xaxis_title="LONGITUDE",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
fig = px.histogram(df,x="LOCATION" , y="VIOLATIONS" ,  color_discrete_sequence=['red'], nbins=10, height=650)
fig.update_layout(
    title_text="Distribution of VIOLATIONS by LOCATION",
    xaxis_title="LOCATION",
    yaxis_title="VIOLATIONS"
)
fig.show()

In [ ]:
df.columns

In [ ]:
df_pivot = df.pivot_table(index='VIOLATION DATE', columns='LATITUDE', values='LONGITUDE', aggfunc='mean').reset_index()
fig = px.area(df_pivot, x='VIOLATION DATE', y=df_pivot.columns[1:], height=400)
fig.update_layout(
    title_text="Distribution of LONGITUDE by LATITUDE Over the VIOLATION DATE",
    xaxis_title="VIOLATION DATE",
    yaxis_title="LONGITUDE",
    legend_title="LATITUDE"
)
fig.show()